# Average Cycle vs Subject Cycles
Compare how similar the moving nodes are in the average cycle vs. subject level cycle. This hints at whether or not
the overlap of all subject level cycles will result in something significant. This will also allow us to use statistical
test and put a number on similarity of subject lvl to average lvl comparisons. For visualizing the statistical ICN changes,
run next block and create Sankey.
- For each ICN, compare the ICN's out going distribution between average cycle and subject level cycle
- [X] Distribution comparison and plot
- [X] Apply distribution comparison to all subject cycles
- [X] Change the subject levels so that we're not doing a simple sum. Instead have each individual switching
distribution and calculate standard deviation across all subjects. How much is the deviation?

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import OAM_functions as ov_fct
import Towlson_group_code.data_io as myFunc
from collections import Counter
from tqdm import tqdm
import importlib
importlib.reload(ov_fct)

fn_colors = {
    0: "#FF3333",
    1: "#66B2FF",
    2: "#B266FF",
    3: "#FFB266",
    4: "#00994C",
    5: "#FFCCFF",
    6: "#F5d414",
    7: "#C90076",
    8: "#E0E0E0"
}

all_cycles = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'all_cycles.pkl')

# Counting Number of Ppl / ICN Switch
To measure the significance of FN behaviors, we use "number of cycles that had this switch" as an
indication of significance. Definitions:

1. FN switch ($fg$): Between EF to LF, 10% of DMN moved to Frontoparietal network.
2. Group switch ($\bar{fg}$): Average the switch % across all subject cycles
3. Counting: we count a cycle's FN switch if for *that cycle*, the % of switch > $\bar{fg}$ + $\sigma_{fg}$,
where $\bar{fg}$ is the group switch.

In [ ]:
all_cycles = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'all_cycles.pkl')
enum = {'EF': 0, 'LF': 1, 'ML': 2}

def get_group_behavior(src_icn, phase_1, phase_2, normalize=True):
    all_counts = []
    for cc in range(30):
        cy = all_cycles[cc]
        if phase_1 == "EF":
            a_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{cy[enum[phase_1]]}_auditory.pkl')
        if phase_1 == "LF":
            a_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{cc}_{phase_1}_partition_auditory_EF.pkl')
        b_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{cc}_{phase_2}_partition_auditory_{phase_1}.pkl')
        all_counts.append(get_behavior(a_partition, b_partition, src_icn, normalize))
    temp_narray = np.array(all_counts)
    return np.mean(temp_narray, axis=0), np.std(temp_narray, axis=0), np.median(temp_narray, axis=0)

def get_behavior(src_partition, target_partition, src_icn, normalize=True):
    de = Counter(src_partition)[src_icn]
    if de == 0:
        return np.array([0]*9)
    partition_zip = list(zip(src_partition, target_partition))
    icount = [0]*9
    for i in range(1054):
        if partition_zip[i][0] == src_icn:    # src ICN
            icount[partition_zip[i][1]] += 1
    if normalize:
        return np.array(icount)/de
    else:
        return np.array(icount)

# Subject Level Representative Sankey
Since we have an ensemble of Sankey, create an artificial Sankey that captures the average of the subject level changes.

*Warning*: Code below is "hard coded" and is not adaptable. This is meant for a 1-time script to estimate group level changes visually.

### Get average ICN sizes

In [ ]:
ef_fn_sizes = []
lf_fn_sizes = []
ml_fn_sizes = []
for cc in range(30):
    if cc in ov_fct.REMOVE:
        continue
    cy = all_cycles[cc]
    pef = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{cy[0]}_auditory.pkl')
    plf = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{cy[1]}_auditory.pkl')
    pml = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{cy[2]}_auditory.pkl')
    for i in range(9):
        ef_fn_sizes.append([Counter(pef)[i] for i in range(9)])
        lf_fn_sizes.append([Counter(plf)[i] for i in range(9)])
        ml_fn_sizes.append([Counter(pml)[i] for i in range(9)])
avg_ef_fn_sizes = np.mean(np.array(ef_fn_sizes), axis=0)
print("EF: ", avg_ef_fn_sizes)
avg_lf_fn_sizes = np.mean(np.array(lf_fn_sizes), axis=0)
print("LF: ", avg_lf_fn_sizes)
avg_ml_fn_sizes = np.mean(np.array(ml_fn_sizes), axis=0)
print("ML: ", avg_ml_fn_sizes)

print("Group Average for EF phase: ")
avg_ef_fn_sizes = np.rint(avg_ef_fn_sizes)
avg_ef_fn_sizes[0] += 1
print(avg_ef_fn_sizes, sum(avg_ef_fn_sizes))

print("Group Average for LF phase: ")
avg_lf_fn_sizes = np.rint(avg_lf_fn_sizes)
# avg_lf_fn_sizes[-1] -= 1
print(avg_lf_fn_sizes, sum(avg_lf_fn_sizes))

print("Group Average for ML phase: ")
avg_ml_fn_sizes = np.rint(avg_ml_fn_sizes)
avg_ml_fn_sizes[0] -= 1
print(avg_ml_fn_sizes, sum(avg_ml_fn_sizes))

group_avg_fn = {"EF": avg_ef_fn_sizes.astype(int), "LF": avg_lf_fn_sizes.astype(int), "ML": avg_ml_fn_sizes.astype(int)}

### Estimate ICN Switching Pattern (No filter)
For EF-LF, LF-ML, EF-ML
- [X] Print out Pattern (for each ICN, largest switch to smallest switch in %)
- [X] Group average (*.pkl)
- [ ] Group median (*_median.pkl)

In [ ]:
old_stdout = sys.stdout
sys.stdout = open('../Ovarian_hormone/Dump/estimated_icn_switching_pattern_no_filter.txt', 'w')

group_avg_ef_partition = []
group_avg_lf_partition = []
print("--------- EF -> LF ---------")
for src_icn in range(9):
    icn_N = int(group_avg_fn["EF"][src_icn])
    group_avg_ef_partition += [src_icn]*icn_N
    grp_behave, grp_err, _ = get_group_behavior(src_icn, "EF", "LF")
    grp_behave_count, _, _ = get_group_behavior(src_icn, "EF", "LF", normalize=False)

    switches = np.rint(grp_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        # print("Need to adjust by ", adj)
        if adj < 0:
            max_diff = np.argmax(grp_behave_count - switches)
            if abs(adj) <= (int(round(grp_behave_count[max_diff])) - int(round(switches[max_diff]))):
                switches[max_diff] += abs(adj);
            else:
                print("UH OH... +")
        else:
            # Remove some switching nodes
            max_diff = np.argmax(switches - grp_behave_count)
            # print(abs(adj), (int(switches[max_diff]) - int(grp_behave_count[max_diff])))
            if abs(adj) <= (int(round(switches[max_diff])) - int(round(grp_behave_count[max_diff]))):
                # print(f"adjusted by {adj} @ {max_diff}")
                switches[max_diff] = switches[max_diff] - adj;
            else:
                print("UH OH... -")
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ")
    for i, s in enumerate(switches):
        print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(s)} nodes) +- {grp_err[i]*100:0.2f}%")
        group_avg_lf_partition += [i]*int(round(s))

# myFunc.save_to_pickle(group_avg_ef_partition, '../Ovarian_hormone/pickles/', 'group_avg_ef_partition.pkl')
# myFunc.save_to_pickle(group_avg_lf_partition, '../Ovarian_hormone/pickles/', 'group_avg_lf_partition.pkl')

print("--------- LF -> ML ---------")
group_avg_lf_partition_ml = []
group_avg_ml_partition = []
for src_icn in range(9):
    icn_N = Counter(group_avg_lf_partition)[src_icn]
    group_avg_lf_partition_ml += [src_icn]*icn_N
    grp_behave, grp_err, _ = get_group_behavior(src_icn, "LF", "ML")
    grp_behave_count, _, _ = get_group_behavior(src_icn, "LF", "ML", normalize=False)

    switches = np.rint(grp_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        if adj < 0:
            max_diff = np.argmax(grp_behave_count - switches)
            # print(adj, max_diff, (int(round(grp_behave_count[max_diff])) - int(round(switches[max_diff]))))
            if abs(adj) <= (int(round(grp_behave_count[max_diff])) - int(round(switches[max_diff]))):
                switches[max_diff] += abs(adj);
            else:
                while adj != 0:
                    max_diff = np.argmax(grp_behave_count - switches)
                    max_adj = abs(int(round(grp_behave_count[max_diff])) - int(round(switches[max_diff])))
                    if max_adj == 0:
                        # print("Nothing left to adj. Just just add everything to unknown network.")
                        switches[8] += abs(adj);
                        adj = 0
                    elif abs(adj) <= max_adj:
                        switches[max_diff] += abs(adj);
                        adj += abs(adj)
                    else:
                        switches[max_diff] += max_adj;
                        adj += max_adj
        else:
            max_diff = np.argmax(switches - grp_behave_count)
            if abs(adj) <= (int(round(switches[max_diff])) - int(round(grp_behave_count[max_diff]))):
                switches[max_diff] = switches[max_diff] - adj;
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ")
    for i, s in enumerate(switches):
        print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(s)} nodes) +- {grp_err[i]*100:0.2f}%")
        group_avg_ml_partition += [i]*int(s)

# myFunc.save_to_pickle(group_avg_ml_partition, '../Ovarian_hormone/pickles/', 'group_avg_ml_partition.pkl')
# myFunc.save_to_pickle(group_avg_lf_partition_ml, '../Ovarian_hormone/pickles/', 'group_avg_lf_partition_ml.pkl')

group_avg_ef_partition = []
group_avg_ml_partition = []
print("--------- EF -> ML ---------")
for src_icn in range(9):
    icn_N = int(group_avg_fn["EF"][src_icn])
    group_avg_ef_partition += [src_icn]*icn_N
    grp_behave, grp_err, _ = get_group_behavior(src_icn, "EF", "ML")
    grp_behave_count, _, _ = get_group_behavior(src_icn, "EF", "ML", normalize=False)

    switches = np.rint(grp_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        # print("Need to adjust by ", adj)
        if adj < 0:
            # Add some switching nodes
            max_diff = np.argmax(grp_behave_count - switches)
            # print(max_diff, (int(round(grp_behave_count[max_diff])) - int(round(switches[max_diff]))))
            if abs(adj) <= (int(round(grp_behave_count[max_diff])) - int(round(switches[max_diff]))):
                switches[max_diff] += abs(adj);
            else:
                while adj != 0:
                    max_diff = np.argmax(grp_behave_count - switches)
                    max_adj = abs(int(round(grp_behave_count[max_diff])) - int(round(switches[max_diff])))
                    if max_adj == 0:
                        # print("Nothing left to adj. Just just add everything to unknown network.")
                        switches[8] += abs(adj);
                        adj = 0
                    elif abs(adj) <= max_adj:
                        switches[max_diff] += abs(adj);
                        adj += abs(adj)
                    else:
                        switches[max_diff] += max_adj;
                        adj += max_adj
        else:
            # Remove some switching nodes
            max_diff = np.argmax(switches - grp_behave_count)
            # print(abs(adj), (int(switches[max_diff]) - int(grp_behave_count[max_diff])))
            if abs(adj) <= (int(round(switches[max_diff])) - int(round(grp_behave_count[max_diff]))):
                # print(f"adjusted by {adj} @ {max_diff}")
                switches[max_diff] = switches[max_diff] - adj;
            else:
                print("UH OH... -")
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ")
    for i, s in enumerate(switches):
        print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(s)} nodes) +- {grp_err[i]*100:0.2f}%")
        group_avg_ml_partition += [i]*int(round(s))
myFunc.save_to_pickle(group_avg_ml_partition, '../Ovarian_hormone/pickles/', 'group_avg_ml_partition_ef.pkl')
sys.stdout = old_stdout

### Estimate ICN Switching Pattern > filter
For EF-LF, LF-ML, EF-ML
- [X] Group avg. > std. dev (*_sig.pkl)
- [X] Subject level > null model
- [X] Print out Pattern (for each ICN, largest switch to smallest switch in %)

In [ ]:
# old_stdout = sys.stdout
# sys.stdout = open('../Ovarian_hormone/Dump/estimated_icn_switching_pattern_sig.txt', 'w')
group_avg_ef_partition = []
group_avg_lf_partition = []
print("--------- EF -> LF ---------")
for src_icn in range(9):
    icn_N = int(group_avg_fn["EF"][src_icn])
    group_avg_ef_partition += [src_icn]*icn_N
    grp_behave, grp_err, grp_median = get_group_behavior(src_icn, "EF", "LF")
    # grp_behave_count, _, _ = get_group_behavior(src_icn, "EF", "LF", normalize=False)
    sig_behave = grp_behave >= grp_err
    for i in range(9):
        if i == src_icn:
            continue
        if not sig_behave[i]:
            grp_behave[src_icn] += grp_behave[i]
            grp_behave[i] = 0

    switches = np.rint(grp_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes) +- {grp_err[i]*100:0.2f}%")
        group_avg_lf_partition += [i]*int(round(s))

# print("EF: ", sorted(Counter(group_avg_ef_partition).items()))
# print("LF: ", sorted(Counter(group_avg_lf_partition).items()))
# print(sum(Counter(group_avg_lf_partition).values()))
# print(sum(Counter(group_avg_ef_partition).values()))
# myFunc.save_to_pickle(group_avg_ef_partition, '../Ovarian_hormone/pickles/', 'group_avg_ef_partition_sig.pkl')
# myFunc.save_to_pickle(group_avg_lf_partition, '../Ovarian_hormone/pickles/', 'group_avg_lf_partition_sig.pkl')

print("--------- LF -> ML ---------")
group_avg_lf_partition_ml = []
group_avg_ml_partition = []
for src_icn in range(9):
    icn_N = Counter(group_avg_lf_partition)[src_icn]
    group_avg_lf_partition_ml += [src_icn]*icn_N
    grp_behave, grp_err, _ = get_group_behavior(src_icn, "LF", "ML")
    # grp_behave_count, _, _ = get_group_behavior(src_icn, "LF", "ML", normalize=False)

    sig_behave = grp_behave >= grp_err
    for i in range(9):
        if i == src_icn:
            continue
        if not sig_behave[i]:
            grp_behave[src_icn] += grp_behave[i]
            grp_behave[i] = 0

    switches = np.rint(grp_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes) +- {grp_err[i]*100:0.2f}%")
            group_avg_ml_partition += [i]*int(s)

# print("LF: ", sorted(Counter(group_avg_lf_partition).items()))
# print("ML: ", sorted(Counter(group_avg_ml_partition).items()))
# print(sum(Counter(group_avg_ml_partition).values()))
# print("Group avg: ", group_avg_fn["ML"])

# myFunc.save_to_pickle(group_avg_ml_partition, '../Ovarian_hormone/pickles/', 'group_avg_ml_partition_sig.pkl')
# myFunc.save_to_pickle(group_avg_lf_partition_ml, '../Ovarian_hormone/pickles/', 'group_avg_lf_partition_ml_sig.pkl')

group_avg_ef_partition = []
group_avg_ml_partition = []
print("--------- EF -> ML ---------")
for src_icn in range(9):
    icn_N = int(group_avg_fn["EF"][src_icn])
    group_avg_ef_partition += [src_icn]*icn_N
    grp_behave, grp_err, _ = get_group_behavior(src_icn, "EF", "ML")
#     grp_behave_count, _, _ = get_group_behavior(src_icn, "EF", "ML", normalize=False)

    sig_behave = grp_behave >= grp_err
    for i in range(9):
        if i == src_icn:
            continue
        if not sig_behave[i]:
            grp_behave[src_icn] += grp_behave[i]
            grp_behave[i] = 0

    switches = np.rint(grp_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes) +- {grp_err[i]*100:0.2f}%")
        group_avg_ml_partition += [i]*int(round(s))

# print(sorted(Counter(group_avg_ef_partition).items()))
# print(sorted(Counter(group_avg_ml_partition).items()))
# print(sum(Counter(group_avg_ml_partition).values()))
# print("Group avg: ", group_avg_fn["ML"])

# myFunc.save_to_pickle(group_avg_ml_partition, '../Ovarian_hormone/pickles/', 'group_avg_ml_partition_ef_sig.pkl')
# sys.stdout = old_stdout

# Null Model
Updated: July 3, 2023

Following methods outlined in Bassett2013a, we'll create a *temporal null model* for the subject level Sankey diagram.
We'll use this null model as a cut-off to measure significance of a community's temporal evolution.

We use a 95% confidence interval for cut-off.

In [ ]:
from random import shuffle

all_cycles = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'all_cycles.pkl')
ef_input_params = myFunc.load_from_pickle("../Ovarian_hormone/pickles/individual_connectomes/",
                                          "ef_input_params.pkl")
ef_name_to_idx = {x[0]: x[1] for x in ef_input_params.values()}

def get_cycle_partitions(ci):
    ef_scan, lf_scan, ml_scan = all_cycles[ci]
    a1 = myFunc.load_from_pickle(f'../Ovarian_hormone/pickles/best_subject_auditory/', ef_scan+'_auditory.pkl')
    a2 = myFunc.load_from_pickle(f'../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{ci}_LF_partition_auditory_EF.pkl')
    a3 = myFunc.load_from_pickle(f'../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{ci}_ML_partition_auditory_LF.pkl')
    return [a1, a2, a3]


def get_switching_pattern(src_partition, target_partition):
    icn_size = Counter(src_partition)
    de = [float(icn_size[i]) if i in icn_size.keys() else 0.0 for i in range(9)]
    icount = np.array([[0.0]*9 for _ in range(9)])
    for i in range(1054):
        icount[src_partition[i]][target_partition[i]] += 1.0
    for i, div in enumerate(de):
        if div == 0.0:
            continue
        icount[i] = icount[i]/div
    return icount

For each cycle, there are 3 phases. Under the null hypothesis, there is no connectivity changes associated with phase.
Therefore we can shuffle the EF, LF, and ML partitions to calculate a null model. Note that we do not do random permutation
because there are at most 6 ways to permute 3 objects (Permute(3,2) = 6). We will account for every permutation of each
cycle to generate the null model.

The average switching pattern for the null model will be averaged over all cycle's permutations (20*6 = 120).

In [ ]:
n = (20)*6
# n = (30)*6
print(n)
null_switching_pattern = np.zeros((n,9,9))
# null_switching_pattern_lf_ml = np.zeros((30000,9,9))
rep_i = 0
n = 0
for c in tqdm(range(30)):
    cycle_partitions = get_cycle_partitions(c)
    if c in ov_fct.REMOVE:
        continue
    for p1, p2 in [(0,1), (1,0), (0,2), (2,0), (2,1), (1,2)]:
        null_switching_pattern[rep_i] = get_switching_pattern(cycle_partitions[p1], cycle_partitions[p2])
        rep_i += 1
        n += 1
print(n)
m, _, ubound = ov_fct.mean_confidence_interval(null_switching_pattern)
null_switching_pattern = ubound
np.fill_diagonal(null_switching_pattern,0)
# print(null_switching_pattern)

Calculate subject level switching pattern

In [ ]:
n = 30 - len(ov_fct.REMOVE)
print("n = ", n)
subject_switching_pattern_ef_lf = np.zeros((n,9,9))
subject_switching_pattern_lf_ml = np.zeros((n,9,9))
subject_switching_pattern_ml_ef = np.zeros((n,9,9))
subject_switching_pattern_ef_ml = np.zeros((n,9,9))
n = 0
for c in tqdm(range(30)):
    if c in ov_fct.REMOVE:
        continue
    cycle_partitions = get_cycle_partitions(c)
    subject_switching_pattern_ef_lf[n] = get_switching_pattern(cycle_partitions[0], cycle_partitions[1])
    subject_switching_pattern_lf_ml[n] = get_switching_pattern(cycle_partitions[1], cycle_partitions[2])
    subject_switching_pattern_ml_ef[n] = get_switching_pattern(cycle_partitions[2], cycle_partitions[0])
    subject_switching_pattern_ef_ml[n] = get_switching_pattern(cycle_partitions[0], cycle_partitions[2])
    n += 1
print(n)
subject_switching_pattern_ef_lf = np.mean(subject_switching_pattern_ef_lf, axis=0)
subject_switching_pattern_lf_ml = np.mean(subject_switching_pattern_lf_ml, axis=0)
subject_switching_pattern_ml_ef = np.mean(subject_switching_pattern_ml_ef, axis=0)
subject_switching_pattern_ef_ml = np.mean(subject_switching_pattern_ef_ml, axis=0)

In [ ]:
print(n)

Plot filtered subject switching pattern for EF - LF transition.

In [ ]:
import seaborn as sns
P = {'Subject': subject_switching_pattern_ef_lf*100, 'Null Model': null_switching_pattern*100,
     'Filtered': subject_switching_pattern_ef_lf*(subject_switching_pattern_ef_lf > null_switching_pattern)*100}

fig, ax = plt.subplots(1,3, figsize=(14,6))
for ax_i, phase in enumerate(['Subject', 'Null Model', 'Filtered']):
    c = ax[ax_i].matshow(P[phase], cmap=sns.color_palette("magma", as_cmap=True), vmin=0, vmax=100)
    ax[ax_i].xaxis.set_ticks_position('bottom')
    ax[ax_i].set_xlabel("Functional Network")
    ax[ax_i].set_title(phase)
    ax[ax_i].set_xticks([i for i in range(9)])
    ax[ax_i].set_xticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)], rotation=45)
    ax[ax_i].set_yticks([i for i in range(9)])
    ax[ax_i].set_yticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)])
    if ax_i == 0:
        ax[ax_i].set_ylabel("Functional Network")

fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.3, 0.02, 0.4])
cbar = fig.colorbar(c, cax=cbar_ax)
cbar.set_label("Switching %", rotation=270, labelpad=15)
# plt.savefig(f"../Ovarian_hormone/Figures/network diagnosis/Null model EF LF.png", dpi=300, bbox_inches='tight')
plt.show()

Plot filtered subject switching pattern for LF - ML transition.

In [ ]:
P = {'Subject': subject_switching_pattern_lf_ml*100, 'Null Model': null_switching_pattern*100,
     'Filtered': subject_switching_pattern_lf_ml*(subject_switching_pattern_lf_ml > null_switching_pattern)*100}

fig, ax = plt.subplots(1,3, figsize=(14,6))
for ax_i, phase in enumerate(['Subject', 'Null Model', 'Filtered']):
    c = ax[ax_i].matshow(P[phase], cmap=sns.color_palette("magma", as_cmap=True), vmin=0, vmax=100)
    ax[ax_i].xaxis.set_ticks_position('bottom')
    ax[ax_i].set_xlabel("Functional Network")
    ax[ax_i].set_xticks([i for i in range(9)])
    ax[ax_i].set_xticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)], rotation=45)
    ax[ax_i].set_yticks([i for i in range(9)])
    ax[ax_i].set_yticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)])
    ax[ax_i].set_title(phase)
    if ax_i == 0:
        ax[ax_i].set_ylabel("Functional Network")
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.3, 0.02, 0.4])
cbar = fig.colorbar(c, cax=cbar_ax)
cbar.set_label("Switching %", rotation=270, labelpad=15)
# plt.savefig(f"../Ovarian_hormone/Figures/network diagnosis/Null model LF ML.png", dpi=300, bbox_inches='tight')
plt.show()

Plot filtered subject switching pattern for EF - ML transition.

In [ ]:
P = {'Subject': subject_switching_pattern_ef_ml*100, 'Null Model': null_switching_pattern*100,
     'Filtered': subject_switching_pattern_ef_ml*(subject_switching_pattern_ef_ml > null_switching_pattern)*100}

fig, ax = plt.subplots(1,3, figsize=(14,6))
for ax_i, phase in enumerate(['Subject', 'Null Model', 'Filtered']):
    c = ax[ax_i].matshow(P[phase], cmap=sns.color_palette("magma", as_cmap=True), vmin=0, vmax=100)
    ax[ax_i].xaxis.set_ticks_position('bottom')
    ax[ax_i].set_xlabel("Functional Network")
    ax[ax_i].set_xticks([i for i in range(9)])
    ax[ax_i].set_xticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)], rotation=45)
    ax[ax_i].set_yticks([i for i in range(9)])
    ax[ax_i].set_yticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)])
    ax[ax_i].set_title(phase)
    if ax_i == 0:
        ax[ax_i].set_ylabel("Functional Network")
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.3, 0.02, 0.4])
cbar = fig.colorbar(c, cax=cbar_ax)
cbar.set_label("Switching %", rotation=270, labelpad=15)
# plt.savefig(f"../Ovarian_hormone/Figures/network diagnosis/Null model EF ML.png", dpi=300, bbox_inches='tight')
plt.show()

Plot filtered subject switching pattern for ML - EF transition.

In [ ]:
P = {'Subject': subject_switching_pattern_ml_ef*100, 'Null Model': null_switching_pattern*100,
     'Filtered': subject_switching_pattern_ml_ef*(subject_switching_pattern_ml_ef > null_switching_pattern)*100}

fig, ax = plt.subplots(1,3, figsize=(14,6))
for ax_i, phase in enumerate(['Subject', 'Null Model', 'Filtered']):
    c = ax[ax_i].matshow(P[phase], cmap=sns.color_palette("magma", as_cmap=True), vmin=0, vmax=100)
    ax[ax_i].xaxis.set_ticks_position('bottom')
    ax[ax_i].set_xlabel("Functional Network")
    ax[ax_i].set_xticks([i for i in range(9)])
    ax[ax_i].set_xticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)], rotation=45)
    ax[ax_i].set_yticks([i for i in range(9)])
    ax[ax_i].set_yticklabels([ov_fct.AVG_EF_FN_ACRYNOMS[i] for i in range(9)])
    ax[ax_i].set_title(phase)
    if ax_i == 0:
        ax[ax_i].set_ylabel("Functional Network")
fig.subplots_adjust(right=0.8)
cbar_ax = fig.add_axes([0.82, 0.3, 0.02, 0.4])
cbar = fig.colorbar(c, cax=cbar_ax)
cbar.set_label("Switching %", rotation=270, labelpad=15)
# plt.savefig(f"../Ovarian_hormone/Figures/network diagnosis/Null model EF ML.png", dpi=300, bbox_inches='tight')
plt.show()

Generate representative subject level Sankey diagram

In [ ]:
group_avg_ef_partition = []
group_avg_lf_partition = []
grp_behave = subject_switching_pattern_ef_lf*(subject_switching_pattern_ef_lf > null_switching_pattern)
# total_n = 0

print("--------- EF -> LF ---------")
for src_icn in range(9):
    icn_N = int(group_avg_fn["EF"][src_icn])
    group_avg_ef_partition += [src_icn]*icn_N
    sig_behave = grp_behave[src_icn,:]
    switches = np.rint(sig_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes)")
        group_avg_lf_partition += [i]*int(round(s))
# myFunc.save_to_pickle(group_avg_ef_partition, '../Ovarian_hormone/pickles/', 'group_avg_20_ef_partition_sig_null_model_25092023.pkl')
# myFunc.save_to_pickle(group_avg_lf_partition, '../Ovarian_hormone/pickles/', 'group_avg_20_lf_partition_sig_null_model_25092023.pkl')

print("--------- LF -> ML ---------")
group_avg_lf_partition_ml = []
group_avg_ml_partition = []
grp_behave = subject_switching_pattern_lf_ml*(subject_switching_pattern_lf_ml > null_switching_pattern)
for src_icn in range(9):
    icn_N = Counter(group_avg_lf_partition)[src_icn]
    group_avg_lf_partition_ml += [src_icn]*icn_N
    sig_behave = grp_behave[src_icn,:]
    switches = np.rint(sig_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes)")
            group_avg_ml_partition += [i]*int(s)
# myFunc.save_to_pickle(group_avg_ml_partition, '../Ovarian_hormone/pickles/', 'group_avg_20_ml_partition_sig_null_model_25092023.pkl')
# myFunc.save_to_pickle(group_avg_lf_partition_ml, '../Ovarian_hormone/pickles/', 'group_avg_20_lf_partition_ml_sig_null_model_25092023.pkl')

print("--------- ML -> EF ---------")
group_avg_ml_partition_ef = []
group_avg_ef_partition_2 = []
grp_behave = subject_switching_pattern_ml_ef*(subject_switching_pattern_ml_ef > null_switching_pattern)
for src_icn in range(9):
    icn_N = Counter(group_avg_ml_partition)[src_icn]
    group_avg_ml_partition_ef += [src_icn]*icn_N
    sig_behave = grp_behave[src_icn,:]
    switches = np.rint(sig_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes)")
            group_avg_ef_partition_2 += [i]*int(s)

myFunc.save_to_pickle(group_avg_ef_partition_2, '../Ovarian_hormone/pickles/', 'group_avg_20_ef_partition_2_sig_null_model_25092023.pkl')
myFunc.save_to_pickle(group_avg_ml_partition_ef, '../Ovarian_hormone/pickles/', 'group_avg_20_ml_partition_ef_sig_null_model_25092023.pkl')

# print("ML similarity = ", sum(np.array([1 if group_avg_ml_partition_ef[i] == group_avg_ml_partition[i] else 0 for i in range(1054)])))

Directly between EF and ML. (not EF - LF - ML)

In [ ]:
group_avg_ef_partition = []
group_avg_ml_partition = []
grp_behave = subject_switching_pattern_ef_ml*(subject_switching_pattern_ef_ml > null_switching_pattern)
print("--------- EF -> ML ---------")
for src_icn in range(9):
    icn_N = int(group_avg_fn["EF"][src_icn])
    group_avg_ef_partition += [src_icn]*icn_N
    sig_behave = grp_behave[src_icn,:]
    switches = np.rint(sig_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes)")
        group_avg_ml_partition += [i]*int(round(s))

myFunc.save_to_pickle(group_avg_ml_partition, '../Ovarian_hormone/pickles/',
                      'group_avg_20_ml_partition_from_ef_sig_null_model_25092023.pkl')

group_avg_ef_partition_2 = []
group_avg_ml_partition_ef = []
grp_behave = subject_switching_pattern_ml_ef*(subject_switching_pattern_ml_ef > null_switching_pattern)
print("--------- ML -> EF ---------")
for src_icn in range(9):
    icn_N = Counter(group_avg_ml_partition)[src_icn]
    group_avg_ml_partition_ef += [src_icn]*icn_N
    sig_behave = grp_behave[src_icn,:]
    switches = np.rint(sig_behave*icn_N)
    if (sum(switches) - icn_N) != 0:
        adj = int(sum(switches) - icn_N)
        switches[src_icn] -= adj
    print(f"{ov_fct.AVG_EF_FN_AUDITORY[src_icn]} Switching Pattern: ", sum(switches), icn_N)
    for i, s in enumerate(switches):
        if int(round(s)) != 0:
            print(f"\t|- {ov_fct.AVG_EF_FN_AUDITORY[i]}: {s*100/icn_N:0.2f}% ({int(round(s))} nodes)")
        group_avg_ef_partition_2 += [i]*int(round(s))

myFunc.save_to_pickle(group_avg_ef_partition_2, '../Ovarian_hormone/pickles/',
                      'group_avg_20_ef_partition_2_direct_sig_null_model_25092023.pkl')
myFunc.save_to_pickle(group_avg_ml_partition_ef, '../Ovarian_hormone/pickles/',
                      'group_avg_20_ml_partition_to_ef_sig_null_model_25092023.pkl')